# Tutorial Convert Solvency 2 XBRL-instances to CSV, HTML and pickles

This tutorial describes how to convert XBRL-instances to csv-, html- and pickle files per template.

In [ ]:
from arelle import ModelManager, Cntlr, ModelFormulaObject, ModelXbrl, ViewFileFormulae, XbrlConst, ViewFileRenderedGrid
from arelle import RenderingEvaluator

In [ ]:
import src
import pandas as pd
from os import listdir, walk, makedirs, environ
from os.path import isfile, join, exists, basename
from datetime import datetime

## Initialize the model manager

In [ ]:
# the taxonomy should be data/taxonomy/arelle
# the instances you want to use should be in data/instances

XBRL_TAXONOMY_PATH = join('..', 'data', 'taxonomy')
XBRL_INSTANCES_PATH = join('..', 'data', 'instances')

LANGUAGE = "en-GB"
environ['XDG_CONFIG_HOME'] = XBRL_TAXONOMY_PATH

In [ ]:
# Now we make a modelmanager
# logFileName = "logToPrint" -> logging is print to notebook
# logFileName = "arelle.log" -> logging is to filename (use .json or .xml for specific format)

controller = Cntlr.Cntlr(logFileName = "logToPrint")
controller.webCache.workOffline = True
controller.logger.messageCodeFilter = None

modelmanager = ModelManager.initialize(controller)
modelmanager.defaultLang = LANGUAGE
modelmanager.formulaOptions = ModelFormulaObject.FormulaOptions()
modelmanager.loadCustomTransforms()

# The role defined in the model.xsd schema for resources representing codes of rows or columns is
euRCcode = 'http://www.eurofiling.info/xbrl/role/rc-code'

## Read XBRL-instance in the modelmanager

In [ ]:
# here we read the XBRL instance
instance_name = 'qrs_240_instance.xbrl'
# instance_name = 'aeb_240_instance.xbrl'

subdir = basename(instance_name).split(".")[0]

In [ ]:
xbrl_instance = ModelXbrl.load(modelmanager, join(XBRL_INSTANCES_PATH, instance_name))
RenderingEvaluator.init(xbrl_instance)

## Convert XBRL-instance to CSV and HTML

In [ ]:
# get tables in instance and sort by short name
tables = list(xbrl_instance.modelRenderingTables)
tables.sort(key = lambda table: table.genLabel(lang = LANGUAGE,strip = True, role = euRCcode))

In [ ]:
# create csv and pickle files
# time_stamp = datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
for table in tables:
    obj = src.generateCSV.generateCSVTables(xbrl_instance, join(XBRL_INSTANCES_PATH, subdir), 
                                            table = table, 
                                            lang = LANGUAGE,
                                            verbose_labels = False)

In [ ]:
# # create html files
# for table in tables:
#     ViewFileRenderedGrid.viewRenderedGrid(xbrl_instance,
#              join(XBRL_INSTANCES_PATH, subdir, table[-13:]+".html"),
#              viewTblELR = table,
#              lang = LANGUAGE, 
#              sourceView = None,
#              diffToFile = False)

## Reading a report from the pickle files

In [ ]:
df = pd.read_pickle(join(XBRL_INSTANCES_PATH, subdir, 'S.02.01.02.01' + ".pickle"))
df

In [ ]:
# from arelle import Validate
# Validate.validate(xbrl_instance)